# Notebook Setup

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

##  CPI Data: Statistics Canada Table 18-10-0004-01  
### *Consumer Price Index (CPI), Monthly, Not Seasonally Adjusted – Converted to Quarterly*

To measure inflation at the provincial level, this project relies on **Statistics Canada Table 18-10-0004-01: Consumer Price Index (CPI)**.  
This is the **primary inflation dataset** in Canada and provides:

- **Monthly CPI values**
- **By province/territory**
- **For both All-Items CPI and 180+ subcomponents**
- **Using 2002=100 or 1992=100 base years depending on the index**

Source (Statistics Canada):  
**18-10-0004-01 – Consumer Price Index, monthly, not seasonally adjusted**  
<https://doi.org/10.25318/1810000401-eng>

### Why This Table Is Important  
This CPI table forms the **core dependent variable** of your analysis. The goal of this capstone project is to understand whether **immigration and housing pressure influence Canadian inflation**, so you need a reliable, long-term measure of CPI across provinces.

The table allows you to:

1. **Construct Quarterly CPI**  
   Because immigration and housing data are quarterly, the monthly CPI is aggregated to quarterly averages:


In [2]:
def process_cpi_data(input_path, output_path, start_year=2010, end_year=None):
    # Load StatsCan data - skipping metadata rows
    df = pd.read_csv(
        input_path, 
        nrows=16, 
        skiprows=[*range(8), 10], 
        on_bad_lines="skip"
    )

    # Fix the header structure: StatsCan often puts dates in columns and Geo in rows 
    # causing a mismatch. We force the first row to be headers.
    geo_headers = df.columns.tolist()
    df.columns = df.iloc[0]
    df = df.drop(index=0).reset_index(drop=True)
    
    # Add the old headers back as a data row before transposing
    geo_row = pd.DataFrame([geo_headers], columns=df.columns)
    df = pd.concat([df, geo_row], ignore_index=True)

    # Transpose to long format: (Month_Year, Geography + CPI series)
    cpi = (
        df.set_index(df.columns[0])
        .T
        .reset_index()
        .rename(columns={"index": "Month_Year"})
    )

    # Clean Geography column
    cpi["Geography"] = cpi["Geography"].astype(str).str.strip()
    cpi["Geography"] = cpi["Geography"].replace(
        to_replace=r"^(Unnamed.*|nan|None|\s*)$", value=pd.NA, regex=True
    )
    cpi["province"] = cpi["Geography"].ffill()
    
    # The '0' column usually holds the reference date string
    cpi = cpi.rename(columns={0: "reference_period"})
    
    # Parse dates
    cpi["date"] = pd.to_datetime(cpi["reference_period"], errors="coerce")
    
    # Check for parse errors
    if cpi["date"].isna().any():
        print(f"Warning: {cpi['date'].isna().sum()} dates failed to parse.")

    # Feature Engineering: Year, Quarter
    cpi["year"] = cpi["date"].dt.year
    cpi["quarter"] = "Q" + cpi["date"].dt.quarter.astype(str)
    cpi["quarter_date"] = cpi["date"].dt.to_period("Q").dt.to_timestamp(how="end")

    # Filter timeframe
    if start_year:
        cpi = cpi[cpi["year"] >= start_year]
    if end_year:
        cpi = cpi[cpi["year"] <= end_year]

    # Clean numeric columns (remove commas and convert)
    cols_to_keep = ["province", "year", "quarter", "quarter_date"]
    # Everything else that isn't metadata is a value column
    value_cols = [c for c in cpi.columns if c not in cols_to_keep + ["Month_Year", "reference_period", "date", "Geography"]]

    for col in value_cols:
        cpi[col] = (
            cpi[col].astype(str)
            .str.replace(",", "", regex=False)
        )
        cpi[col] = pd.to_numeric(cpi[col], errors="coerce")

    # Aggregation: Monthly -> Quarterly Mean
    cpi_quarterly = (
        cpi.groupby(cols_to_keep)[value_cols]
        .mean(numeric_only=True)
        .reset_index()
    )

    # Cleanup column names (remove footnote markers like "Shelter 10")
    cpi_quarterly.columns = cpi_quarterly.columns.str.replace(r"\s*\d+$", "", regex=True)
    
    # Save output
    cpi_quarterly = cpi_quarterly.sort_values(["province", "quarter_date"])
    
    out_dir = Path(output_path).parent
    if not out_dir.exists():
        out_dir.mkdir(parents=True, exist_ok=True)
        
    cpi_quarterly.to_csv(output_path, index=False)
    print(f"Successfully saved quarterly CPI data to {output_path}")

    return cpi_quarterly

In [3]:
# Define paths
raw_path = "../data/raw/1810000401.csv" 
output_path = "../data/structured/cpi_structured.csv"

# Run processing
df_cpi = process_cpi_data(raw_path, output_path)

print(f"Saved to: {output_path}")
print(f"Shape: {df_cpi.shape}")

# Quick check on feature count (subtracting the 4 metadata cols: province, year, etc)
print(f"CPI Categories: {len(df_cpi.columns) - 4}")

df_cpi.head()

Successfully saved quarterly CPI data to ../data/structured/cpi_structured.csv
Saved to: ../data/structured/cpi_structured.csv
Shape: (1890, 19)
CPI Categories: 15


C:\Users\darre\AppData\Local\Temp\ipykernel_21556\635750614.py:39: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cpi["date"] = pd.to_datetime(cpi["reference_period"], errors="coerce")


Products and product groups 3 4,province,year,quarter,quarter_date,All-items,Food,Shelter,"Household operations, furnishings and equipment",Clothing and footwear,Transportation,Gasoline,Health and personal care,"Recreation, education and reading","Alcoholic beverages, tobacco products and recreational cannabis",All-items excluding food and energy,All-items excluding energy,Energy,Goods,Services
0,Alberta,2010,Q1,2010-03-31 23:59:59.999999999,122.366667,121.700000,147.800000,107.133333,96.600000,118.100000,141.166667,119.766667,105.533333,133.466667,120.200000,120.433333,144.766667,109.133333,136.500000
1,Alberta,2010,Q2,2010-06-30 23:59:59.999999999,122.566667,121.766667,146.133333,107.833333,97.000000,119.566667,143.366667,121.266667,106.333333,133.700000,120.966667,121.100000,138.933333,108.533333,137.466667
2,Alberta,2010,Q3,2010-09-30 23:59:59.999999999,122.866667,121.800000,148.500000,107.466667,95.266667,118.033333,142.900000,122.366667,106.966667,133.566667,120.633333,120.800000,146.166667,108.566667,138.033333
3,Alberta,2010,Q4,2010-12-31 23:59:59.999999999,122.866667,122.033333,147.966667,108.366667,95.133333,119.600000,145.900000,122.200000,104.966667,134.066667,121.033333,121.166667,141.966667,108.566667,138.100000
4,Alberta,2011,Q1,2011-03-31 23:59:59.999999999,124.066667,123.666667,149.600000,108.600000,94.633333,122.833333,163.733333,122.533333,104.366667,134.233333,121.033333,121.466667,155.433333,110.500000,138.500000


## International Migration Components  
### Statistics Canada Table 17-10-0040-01  
**Estimates of the Components of International Migration, Quarterly**

This dataset provides the **core international migration indicators** required to understand how global population flows affect provincial demographic pressure. It is one of the central data sources in the project.

**Official Source**  
Statistics Canada, *Table 17-10-0040-01 — Estimates of the components of international migration, quarterly*  
DOI: **10.25318/1710004001-eng**  
<https://doi.org/10.25318/1710004001-eng>

---

### What the Table Contains

The table reports **quarterly international migration activity** for each province and territory, including:

| Variable | Description |
|----------|-------------|
| **Immigrants** | Number of permanent residents admitted to the province |
| **Emigrants** | Permanent residents who leave Canada to settle abroad |
| **Net_Emigration** | Emigrants − Returning emigrants |
| **Returning_Emigrants** | Canadians who return after living abroad |
| **Net_Temporary_Emigration** | Temporary outflows (students, workers leaving Canada) |
| **Net_Non_Permanent_Residents (NPR)** | Net change in temporary residents (workers, students, asylum claimants) |
| **NPR_Inflows** | Inbound temporary residents |
| **NPR_Outflows** | Temporary residents leaving the province |

These indicators create a complete picture of population change due solely to **international factors**.

---

### Why This Table Is Essential for the Project

International migration is one of the **primary explanatory variables** in the CPI–immigration modeling framework. This table supports the project in several key ways:

#### **1. Measuring Population Shocks**
Immigrants and NPRs represent sudden additions to the population.  
These shocks influence:

- **housing demand**  
- **labour market dynamics**  
- **provincial consumption patterns**  
- **pressure on services**

All of which feed directly into inflation metrics — especially **Shelter CPI** and **Services CPI**.

#### **2. Building High-Value Exogenous Features**
Using this dataset, we construct several features:

```python
immigration_growth_rate
immigration_per_capita
net_migration_rate_per_1000
housing_pressure = Immigrants / total_units


In [4]:
def process_immigration_data(input_path, output_path):
    # Load StatsCan data - skipping metadata rows
    df = pd.read_csv(input_path, skiprows=7)
    
    # We expect 10 specific columns. 
    # Grab the first 10 columns regardless of their current messy names.
    df = df.iloc[:, :10]
    
    df.columns = [
        "province", "reference_period", "immigrants", "net_emigration", 
        "emigrants", "returning_emigrants", "net_temp_emigration",
        "net_npr", "npr_inflows", "npr_outflows"
    ]
    
    # Forward fill province names (handles merged cells in source)
    df["province"] = df["province"].ffill()
    
    # Clean numeric columns (remove commas, handle ".." missing values)
    numeric_cols = df.columns[2:]
    for col in numeric_cols:
        df[col] = pd.to_numeric(
            df[col].astype(str).str.replace(",", "").replace("..", np.nan),
            errors="coerce"
        )

    # Extract Year and Quarter from "reference_period" (e.g., "Q1 2015")
    # This regex looks for: (Q + digit) space (4 digits)
    date_parts = df["reference_period"].astype(str).str.extract(r"(Q\d)\s+(\d{4})")
    
    # Filter valid rows: If we couldn't find a date, it's likely a footer/header row
    valid_rows = date_parts[0].notna()
    df = df.loc[valid_rows].copy()
    
    df["quarter"] = date_parts.loc[valid_rows, 0]
    df["year"] = date_parts.loc[valid_rows, 1].astype(int)

    # Convert Quarters to actual timestamps (Month End)
    # Q1 -> March, Q2 -> June, etc.
    q_map = {"Q1": 3, "Q2": 6, "Q3": 9, "Q4": 12}
    
    df["quarter_date"] = pd.to_datetime({
        "year": df["year"],
        "month": df["quarter"].map(q_map),
        "day": 1
    }) + pd.offsets.MonthEnd(0)
    
    # Final cleanup
    df = df.sort_values(["province", "quarter_date"]).reset_index(drop=True)
    
    # Save
    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(output_path, index=False)
    
    print(f"Processed immigration data saved to {output_path}")
    return df

In [5]:
imm_raw_path = "../data/raw/1710004001-eng.csv"
imm_output_path = "../data/structured/immigration_structured.csv"

# Process
df_immigration = process_immigration_data(imm_raw_path, imm_output_path)

print(f"Saved to: {imm_output_path}")
print(f"Shape: {df_immigration.shape}")

# Quick sanity check
print(f"Unique Provinces: {df_immigration['province'].nunique()}")
print(f"Date Range: {df_immigration['quarter_date'].min()} to {df_immigration['quarter_date'].max()}")

df_immigration.head()

Processed immigration data saved to ../data/structured/immigration_structured.csv
Saved to: ../data/structured/immigration_structured.csv
Shape: (930, 13)
Unique Provinces: 15
Date Range: 2010-03-31 00:00:00 to 2025-06-30 00:00:00


,province,reference_period,immigrants,net_emigration,emigrants,returning_emigrants,net_temp_emigration,net_npr,npr_inflows,npr_outflows,quarter,year,quarter_date
0,Alberta,Q1 2010,6765.0,1313.0,1450.0,668.0,531.0,-141.0,NaN,NaN,Q1,2010,2010-03-31
1,Alberta,Q2 2010,9732.0,1082.0,1731.0,1211.0,562.0,-647.0,NaN,NaN,Q2,2010,2010-06-30
2,Alberta,Q3 2010,9485.0,825.0,2169.0,2002.0,658.0,-1794.0,NaN,NaN,Q3,2010,2010-09-30
3,Alberta,Q4 2010,6675.0,1167.0,1530.0,907.0,544.0,-6587.0,NaN,NaN,Q4,2010,2010-12-31
4,Alberta,Q1 2011,5917.0,1284.0,1617.0,878.0,545.0,-220.0,NaN,NaN,Q1,2011,2011-03-31


##  Interprovincial Migration Components  
### Statistics Canada Table 17-10-0020-01  
**Estimates of the Components of Interprovincial Migration, Quarterly**

This dataset provides detailed quarterly estimates of **interprovincial population movements within Canada**. These flows measure how Canadians relocate between provinces, creating shifts in local demand, labour availability, and housing pressure—factors that directly shape inflation, especially **Shelter CPI**.

 **Official Source**  
Statistics Canada, *Table 17-10-0020-01 — Estimates of the components of interprovincial migration, quarterly*  
DOI: **10.25318/1710002001-eng**  
<https://doi.org/10.25318/1710002001-eng>

---

###  What the Table Contains

The table provides quarterly estimates for each province and territory on:

| Variable | Description |
|----------|-------------|
| **in_migrants** | Number of people moving *into* the province from another Canadian province |
| **out_migrants** | Number of people moving *out* of the province to another Canadian province |
| **net_interprovincial_migration** | Net provincial migration = in_migrants − out_migrants |
| **reference_period_interprov** | The quarter associated with the migration count |

These flows measure **internal population redistribution**, which complements international migration data.

---

###  Why This Table Matters for the Project

Inflation doesn’t respond only to international arrivals — Canadians relocating across provinces play a massive role in reshaping demand and housing markets. This dataset enhances the model by providing:

#### **1. A Localized View of Demand Pressure**
Interprovincial inflows directly raise demand for:

- housing  
- transportation  
- local services  
- food and essentials  

This often produces immediate upward pressure on CPI components (especially rent, shelter, services).

#### **2. Structural Adjustment Signals**
If a province consistently loses population to others, it may experience:

- weaker labour markets  
- slower consumption growth  
- lower demand-driven inflation

Conversely, high-gain provinces (e.g., Alberta during oil booms) show strong CPI responses.

#### **3. Improved Accuracy of Population-Based Features**
With this table, we can compute:

```python
net_interprovincial_rate = net_interprovincial_migration / population * 1000
total_population_pressure = Immigrants + Net_NPR + net_interprovincial_migration


In [6]:
def process_interprovincial_migration(input_path, output_path):
    # Load raw data
    df = pd.read_csv(input_path, skiprows=7)
    
    # Rename columns
    df = df.rename(columns={
        "Unnamed: 0": "province",
        "Interprovincial migration": "reference_period",
        "In-migrants": "in_migrants",
        "Out-migrants": "out_migrants"
    })
    
    # Clean Province (remove footnote digits)
    df["province"] = df["province"].astype(str).str.replace(r"\s+\d.*$", "", regex=True).str.strip()

    # Clean Numeric Columns
    cols_to_clean = ["in_migrants", "out_migrants"]
    for col in cols_to_clean:
        df[col] = pd.to_numeric(
            df[col].astype(str).str.replace(",", "").replace(["None", "..", "nan"], np.nan),
            errors="coerce"
        )
        
    # Parse Dates (Reference Period: "Q1 2015")
    date_parts = df["reference_period"].astype(str).str.extract(r"(Q\d)\s+(\d{4})")
    
    # Filter valid rows (removes metadata/footers automatically)
    mask = date_parts[0].notna()
    df = df.loc[mask].copy()
    
    df["quarter"] = date_parts.loc[mask, 0]
    df["year"] = date_parts.loc[mask, 1].astype(int)

    # Convert Quarter -> Month End Date
    q_map = {"Q1": 3, "Q2": 6, "Q3": 9, "Q4": 12}
    
    df["quarter_date"] = pd.to_datetime({
        "year": df["year"],
        "month": df["quarter"].map(q_map),
        "day": 1
    }) + pd.offsets.MonthEnd(0)
    
    # Calculate Net Migration
    df["net_interprovincial_migration"] = df["in_migrants"] - df["out_migrants"]

    # Sort
    df = df.sort_values(["province", "quarter_date"]).reset_index(drop=True)
    
    # Save
    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(output_path, index=False)
    
    print(f"Interprovincial migration data saved to {output_path}")
    return df

In [7]:
interprov_raw = "../data/raw/1710002001-eng.csv"  
interprov_out = "../data/structured/interprovincial_migration_structured.csv"

# Run processing
df_interprov = process_interprovincial_migration(interprov_raw, interprov_out)

print(f"Saved to: {interprov_out}")
print(f"Shape: {df_interprov.shape}")

# Sanity check on the calculated net migration
df_interprov[['province', 'quarter_date', 'net_interprovincial_migration']].head()

Interprovincial migration data saved to ../data/structured/interprovincial_migration_structured.csv
Saved to: ../data/structured/interprovincial_migration_structured.csv
Shape: (930, 8)


,province,quarter_date,net_interprovincial_migration
0,Alberta,2010-03-31,148.0
1,British Columbia,2010-03-31,1613.0
2,Canada,2010-03-31,0.0
3,Manitoba,2010-03-31,-224.0
4,New Brunswick,2010-03-31,243.0


## Population Data: Statistics Canada Table 17-10-0005-01  
### *Estimates of Population, Quarterly, by Province and Territory*

This project uses population estimates from **Statistics Canada Table 17-10-0005-01** (formerly 1710004001), which provides **quarterly population totals for every province and territory in Canada**.

Source (Statistics Canada):  
**17-10-0005-01 — Estimates of population, quarterly**  
<https://doi.org/10.25318/1710000501-eng>

### Why This Table Is Important  
Population is a **core structural variable** in this project. It provides the denominator needed to transform immigration flows into meaningful, comparable intensity metrics.

The raw number of immigrants entering a province does *not* capture economic pressure. For example:

- 10,000 immigrants in **Ontario** is small relative to its population  
- 10,000 immigrants in **PEI** is enormous  

Therefore, population estimates are essential to convert immigration into **pressure-based indicators** that reflect real economic impact.

###  Key Contributions of This Table

#### **1. Creation of Per-Capita Immigration Variables**
Using provincial population, we compute:
```python
immigration_per_capita = Immigrants / population


In [8]:
def process_population_data(input_path, output_path, start_year=2010):
    # Load raw data
    df = pd.read_csv(input_path, skiprows=7)
    
    # Rename the first column (date)
    df = df.rename(columns={df.columns[0]: "reference_period"})
    
    # Melt: Convert from Wide (Provinces as columns) to Long
    df = df.melt(
        id_vars="reference_period", 
        var_name="province", 
        value_name="population"
    )
    
    # Extract Quarter and Year
    # Regex looks for (Q + digit) + space + (4 digits)
    date_parts = df["reference_period"].astype(str).str.extract(r"(Q\d)\s+(\d{4})")
    
    # Drop rows where date parsing failed (removes footers/metadata)
    valid_rows = date_parts[0].notna()
    df = df.loc[valid_rows].copy()
    
    df["quarter"] = date_parts.loc[valid_rows, 0]
    df["year"] = date_parts.loc[valid_rows, 1].astype(int)
    
    # Convert Quarter to Date
    q_map = {"Q1": 3, "Q2": 6, "Q3": 9, "Q4": 12}
    
    df["quarter_date"] = pd.to_datetime({
        "year": df["year"],
        "month": df["quarter"].map(q_map),
        "day": 1
    }) + pd.offsets.MonthEnd(0)
    
    # Clean Population Values (remove commas)
    df["population"] = pd.to_numeric(
        df["population"].astype(str).str.replace(",", ""),
        errors="coerce"
    )
    
    # Filter by start year
    if start_year:
        df = df[df["year"] >= start_year]
        
    # Sort and Save
    df = df.sort_values(["province", "quarter_date"]).reset_index(drop=True)
    
    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(output_path, index=False)
    
    print(f"Population data saved to {output_path}")
    return df

In [9]:
pop_raw = "../data/raw/1710000901-eng.csv"
pop_out = "../data/structured/population_structured.csv"

# Process
df_population = process_population_data(pop_raw, pop_out)

print(f"Saved to: {pop_out}")
print(f"Shape: {df_population.shape}")

# Quick sanity check on the numbers (e.g. for Ontario)
# It's good to verify if these are raw counts (14,000,000) or thousands (14,000)
display(df_population[df_population['province'].str.contains("Ontario")].tail(3))

Population data saved to ../data/structured/population_structured.csv
Saved to: ../data/structured/population_structured.csv
Shape: (882, 6)


,reference_period,province,population,quarter,year,quarter_date
627,Q1 2025,Ontario,16255550,Q1,2025,2025-03-31
628,Q2 2025,Ontario,16256538,Q2,2025,2025-06-30
629,Q3 2025,Ontario,16258260,Q3,2025,2025-09-30


## Housing Supply Dynamics  
### Statistics Canada Table 34-10-0135-01  
**Canada Mortgage and Housing Corporation (CMHC): Housing Starts, Units Under Construction, and Completions — Quarterly**

This dataset provides comprehensive quarterly housing supply indicators from the Canada Mortgage and Housing Corporation (CMHC). It captures how many new homes are being started, built, and completed across all provinces and territories — a critical factor for understanding *supply-side pressures* on shelter costs and inflation.

 **Official Source**  
Statistics Canada, *Table 34-10-0135-01 — Canada Mortgage and Housing Corporation, housing starts, under construction and completions, all areas, quarterly*  
DOI: **10.25318/3410013501-eng**  
<https://doi.org/10.25318/3410013501-eng>

---

### What the Table Contains

This table reports several core indicators of Canada’s housing stock:

| Feature | Description |
|--------|-------------|
| **total_units** | Total housing starts for the quarter (all dwelling types) |
| **single_detached** | Count of single-detached housing starts |
| **multiples** | Apartments, condos, and semi-condensed multi-unit starts |
| **semi_detached** | Semi-detached unit starts |
| **row_units** | Row house starts |
| **apartment_other** | Other apartment-style units under construction or completed |
| **reference_period_housing** | Quarter label from the source table |

These categories allow us to break down the housing supply by type and identify differences across provinces.

---

###Why This Table Matters for the Project

Housing supply is the **most important mediator** between population growth and shelter inflation. This dataset provides the structural supply variables necessary to measure *housing pressure* and *construction responsiveness*.

#### **1. Measures Construction Response to Population Growth**
The model can evaluate:

- Does housing supply keep up with immigration?
- Do provinces with more housing starts experience lower shelter inflation?

This is essential for understanding persistent affordability gaps.

#### **2. Creates the “Housing Pressure” Metric**
Using this dataset, we compute:

```python
housing_pressure = Immigrants / total_units


In [23]:
def process_housing_starts(input_path, output_path, start_year=2010):
    # Load raw data
    df = pd.read_csv(input_path, skiprows=7)
    
    unit_cols = df.iloc[0, 3:].tolist()
    
    # Drop the header row from the data
    df = df.iloc[1:].copy()

    # Assign clean initial column names
    df.columns = ["province", "seasonal_adjustment", "reference_period"] + unit_cols

    # Clean text columns
    df["province"] = df["province"].ffill()
    df["seasonal_adjustment"] = df["seasonal_adjustment"].astype(str).str.strip()

    # Parse Dates: Find rows like "Q1 2010"
    date_parts = df["reference_period"].astype(str).str.extract(r"(Q[1-4])\s+(\d{4})")
    
    # Keep only valid data rows
    mask = date_parts[0].notna()
    df = df.loc[mask].copy()

    df["quarter"] = date_parts.loc[mask, 0]
    df["year"] = date_parts.loc[mask, 1].astype(int)

    # Convert Quarter -> Date
    q_map = {"Q1": 3, "Q2": 6, "Q3": 9, "Q4": 12}
    df["quarter_date"] = pd.to_datetime({
        "year": df["year"],
        "month": df["quarter"].map(q_map),
        "day": 1
    }) + pd.offsets.MonthEnd(0)

    # Clean Numeric Columns
    for col in unit_cols:
        df[col] = pd.to_numeric(
            df[col].astype(str).str.replace(",", "").replace(["nan", "..", ""], np.nan),
            errors="coerce"
        )

    # Filter by Year
    if start_year:
        df = df[df["year"] >= start_year]

    # Rename housing categories to snake_case for easy modeling later
    rename_map = {
        "Total units": "total_housing_starts",
        "Single-detached": "single_detached_starts",
        "Multiples": "multiples_starts",
        "Semi-detached": "semi_detached_starts",
        "Row": "row_housing_starts",
        "Apartment and other unit type": "apartment_starts"
    }
    # Only rename columns that actually exist in the file
    df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})

    # Final Polish
    cols_to_drop = ["seasonal_adjustment", "reference_period"] # No longer needed
    df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])
    
    df = df.sort_values(["province", "quarter_date"]).reset_index(drop=True)

    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(output_path, index=False)
    
    print(f"Housing starts saved to {output_path}")
    return df

In [24]:
housing_raw_path = "../data/raw/3410013501.csv"
housing_output_path = "../data/structured/housing_starts_quarterly.csv"

# Process
housing_q = process_housing_starts(housing_raw_path, housing_output_path, start_year=2010)

print(f"Shape: {housing_q.shape}")
display(housing_q.head())

Housing starts saved to ../data/structured/housing_starts_quarterly.csv
Shape: (819, 10)


,province,total_housing_starts,single_detached_starts,multiples_starts,semi_detached_starts,row_housing_starts,apartment_starts,quarter,year,quarter_date
0,Alberta,5651.0,3976.0,1675.0,749.0,436.0,490.0,Q1,2010,2010-03-31
1,Alberta,8238.0,5580.0,2658.0,743.0,824.0,1091.0,Q2,2010,2010-06-30
2,Alberta,7437.0,4639.0,2798.0,683.0,565.0,1550.0,Q3,2010,2010-09-30
3,Alberta,5762.0,3656.0,2106.0,562.0,771.0,773.0,Q4,2010,2010-12-31
4,Alberta,4211.0,2462.0,1749.0,466.0,485.0,798.0,Q1,2011,2011-03-31


##  Monetary Policy & Interest Rates  
### Statistics Canada Table 10-10-0139-01  
**Bank of Canada — Money Market and Other Interest Rates (Monthly/Quarterly)**

This dataset provides key interest rate indicators published by the Bank of Canada, including policy-driven rates and market-based yields. These variables influence household borrowing costs, mortgage rates, business investment, and broader inflation dynamics.

 **Official Source**  
Statistics Canada, *Table 10-10-0139-01 — Bank of Canada, money market and other interest rates*  
DOI: **10.25318/1010013901-eng**  
<https://doi.org/10.25318/1010013901-eng>

---

### What the Table Contains

This dataset typically includes:

| Feature | Description |
|--------|-------------|
| **target_rate** | Bank of Canada’s policy interest rate (overnight target) |
| **deposit_rate** | Deposit rate used by financial institutions |
| **bank_rate** | Rate at which banks borrow from the Bank of Canada |
| **treasury_bill_yield** | Short-term (3-month) government T-bill yields |
| **commercial_paper_rate** | Business funding cost indicator |
| **bond_yields (1y–10y)** | Yield curve measures influencing mortgages |

*(Column names may differ depending on your extraction; in your dataset, `target_rate` is already included.)*

---

### Why This Table Matters for the Project

Interest rates form the **monetary-policy backbone** of inflation movements. For shelter inflation especially, rate changes strongly affect mortgage costs and rental markets.

#### **1. Controls for Monetary Policy Shocks**
Including interest rates lets the model isolate:

- how much of CPI movement is due to immigration  
- vs. how much is due to Bank of Canada policy

This improves causality.

#### **2. Helps Interpret Shelter CPI**
Shelter costs are influenced by:

- mortgage interest costs  
- financing for new housing development  
- landlord borrowing costs  

Adding `target_rate` provides a macro-level lens.


---

### Role in the Econometric Model

Interest rate variables can be incorporated in multiple ways:

- As an **exogenous variable** to explain short-term inflation pressure  
- As a **control variable** to test whether immigration retains significance after policy shifts  
- As part of a **hybrid baseline model** with CPI category-level predictors  

Possible engineered features:

```python
rate_change = target_rate.diff()
is_rate_hike = (rate_change > 0).astype(int)
real_rate = target_rate - inflation_rate


In [25]:
def process_boc_target_rate(input_path, output_path, start_year=2010):
    # Load raw data 
    df_raw = pd.read_csv(input_path, skiprows=7, header=None)

    # Dynamic Header Search: Find row where first column is "Geography"
    # This is safer than hardcoding row numbers
    header_idx = df_raw[df_raw[0].astype(str).str.contains("Geography", na=False)].index[0]
    
    # Set header and slice data
    df = df_raw.iloc[header_idx + 2:].copy() # +2 skips the header itself + unit row
    df.columns = df_raw.iloc[header_idx]
    
    # Rename basic columns
    df = df.rename(columns={
        df.columns[0]: "geography",
        df.columns[1]: "reference_period",
        df.columns[-1]: "target_rate"
    })

    # Filter for Canada and clean Rate
    df = df[df["geography"].fillna("").str.contains("Canada")]
    
    # Parse Dates
    df["date"] = pd.to_datetime(df["reference_period"], errors="coerce")
    df = df.dropna(subset=["date"])

    # Clean numeric Rate
    df["target_rate"] = pd.to_numeric(
        df["target_rate"].astype(str).replace(["..", "nan"], pd.NA), 
        errors="coerce"
    )

    # Feature Engineering: Year, Quarter
    df["year"] = df["date"].dt.year
    df["quarter"] = "Q" + df["date"].dt.quarter.astype(str)
    
    # Filter Year
    if start_year:
        df = df[df["year"] >= start_year]

    # Aggregation: Daily to Quarterly Mean
    # We aggregate by Year/Quarter first
    df_quarterly = (
        df.groupby(["year", "quarter"])["target_rate"]
        .mean()
        .reset_index()
    )

    # Add Quarter End Date (for merging with other datasets)
    # Using 'Q' offset to get the last day of the quarter
    df_quarterly["quarter_date"] = (
        pd.to_datetime(df_quarterly["year"].astype(str) + df_quarterly["quarter"])
        + pd.offsets.QuarterEnd(0)
    )

    provinces = [
        "Alberta", "British Columbia", "Manitoba", "New Brunswick", 
        "Newfoundland and Labrador", "Nova Scotia", "Ontario", 
        "Prince Edward Island", "Quebec", "Saskatchewan"
    ]
    
    dfs = []
    for prov in provinces:
        temp = df_quarterly.copy()
        temp["province"] = prov
        dfs.append(temp)
    
    df_final = pd.concat(dfs, ignore_index=True)

    # Sort 
    df_final = df_final[["province", "quarter_date", "year", "quarter", "target_rate"]]
    df_final = df_final.sort_values(["province", "quarter_date"])

    # Save
    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    df_final.to_csv(output_path, index=False)

    print(f"Interest Rate data saved to {output_path}")
    return df_final

In [76]:
boc_raw_path = "../data/raw/1010013901-eng.csv"  
boc_output_path = "../data/structured/boc_target_rate_quarterly.csv"

# Process
boc_q = process_boc_target_rate(boc_raw_path, boc_output_path, start_year=2010)

print(f"Shape: {boc_q.shape}")
display(boc_q.head())

Interest Rate data saved to: ../data/structured/boc_target_rate_quarterly.csv
Shape: (704, 5)
Shape: (704, 5)


,province,year,quarter,quarter_date,target_rate
0,Alberta,2010,Q1,2010-03-31,0.250000
1,Alberta,2010,Q2,2010-06-30,0.334615
2,Alberta,2010,Q3,2010-09-30,0.765152
3,Alberta,2010,Q4,2010-12-31,1.000000
4,Alberta,2011,Q1,2011-03-31,1.000000


## Master Panel Builder — Logic Overview

To create a unified quarterly panel dataset, we use a custom function `build_master_panel()` that loads, standardizes, and merges all source tables (CPI, immigration, migration, housing, population).

This ensures a **consistent structure**, **aligned time index**, and **clean province keys** across all datasets.

---

###  Standardizing Each Dataset
Every file is first run through `standardize_panel()` to:

- **Fix province names**  
  (e.g., “Nfld. and Labrador” → “Newfoundland and Labrador”)  
  ensuring all sources can join cleanly.

- **Convert dates into quarterly timestamps**  
  - parse `quarter_date`  
  - convert to `Period('Q')`  
  - convert back to end-of-quarter timestamp  
  - derive `year` and `quarter`  

This gives all datasets a **uniform temporal structure**, which is required for merging.

---

###  Controlled Multi-Source Merge
The function uses **population** as the foundation, since it always contains a complete province × quarter grid.

Then each dataset is merged **one by one** using the join keys:



In [77]:
def build_master_panel(files_dict, output_path):
    
    def standardize_panel(df, name):
        df = df.copy()

        # --- Province standardization ---
        if "province" in df.columns:
            df["province"] = df["province"].astype(str).str.strip()

            province_map = {
                "Prince Edward Isl.": "Prince Edward Island",
                "Nfld. and Labrador": "Newfoundland and Labrador",
                "Newfoundland and labrador": "Newfoundland and Labrador",
            }
            df["province"] = df["province"].replace(province_map)

        # --- Standardize quarter_date + derive year/quarter ---
        if "quarter_date" in df.columns:
            df["quarter_date"] = (
                pd.to_datetime(df["quarter_date"], errors="coerce")
                .dt.to_period("Q")
                .dt.to_timestamp(how="end")
                .dt.normalize()
            )

            df["year"] = df["quarter_date"].dt.year.astype("Int64")
            df["quarter"] = "Q" + df["quarter_date"].dt.quarter.astype(str)

        return df

   
    # LOAD & STANDARDIZE
   
    data_frames = {}
    print(" Loading datasets and standardizing keys...")

    for key, path in files_dict.items():
        if Path(path).exists():
            df = pd.read_csv(path, low_memory=False)

            df = standardize_panel(df, key)
            data_frames[key] = df

            print(f"    Loaded {key}: {df.shape}")
        else:
            print(f"    File not found: {key}. Skipping.")

  
    # MERGE LOGIC
  
    def merge_datasets(dfs):
        if "population" not in dfs:
            raise ValueError("CRITICAL: Population data missing. Cannot build master.")

        master = dfs["population"].copy()
        merge_order = [k for k in dfs.keys() if k != "population"]

        join_keys = ["province", "quarter_date", "year", "quarter"]

        print("\n Merging...")

        for key in merge_order:
            df_to_merge = dfs[key]

            # Ensure all join keys exist
            if not all(col in df_to_merge.columns for col in join_keys):
                print(f"    Skipping {key}: Missing join keys {join_keys}")
                continue

            master = pd.merge(
                master,
                df_to_merge,
                on=join_keys,
                how="left",
                suffixes=("", f"_{key}")
            )
            print(f"   + Joined {key}. New shape: {master.shape}")

        return master

    
    # CREATE MASTER PANEL
 
    df_final = merge_datasets(data_frames)

   
    #  FINAL CLEANUP

    cols_to_drop = [
        'seasonal_adjustment',
        'Reference_Period',
        'date',
        'Geography'
    ]

    df_final = df_final.drop(columns=cols_to_drop, errors='ignore')

    # Remove unnamed garbage columns
    df_final = df_final.loc[:, ~df_final.columns.str.contains('^Unnamed')]

    # Final sorting
    df_final = df_final.sort_values(["province", "quarter_date"]).reset_index(drop=True)

  
    #  SAVE BOTH:
    #    - output_path (your provided filename)
    #    - master_dataset.csv (gold standard)
 
    Path(output_path).parent.mkdir(parents=True, exist_ok=True)
    df_final.to_csv(output_path, index=False)

    gold_path = "../data/processed/master_dataset.csv"
    Path(gold_path).parent.mkdir(parents=True, exist_ok=True)
    df_final.to_csv(gold_path, index=False)

    print("\n MASTER DATASET CREATED SUCCESSFULLY")
    print(f" • Saved to: {output_path}")
    print(f" • Gold Standard Saved to: {gold_path}")
    print(f" • Final Dimensions: {df_final.shape}")

    print("\nTop 5 Rows (Ontario):")
    print(df_final[df_final['province'] == 'Ontario'].head())

    return df_final


In [93]:
FILES = {
    "population": "../data/structured/population_structured.csv",
    "immigration": "../data/structured/immigration_structured.csv",
    "interprov": "../data/structured/interprovincial_migration_structured1.csv",
    "cpi": "../data/structured/cpi_structured.csv",
    "interest_rate": "../data/structured/boc_target_rate_quarterly.csv",
    "housing": "../data/structured/housing_starts_quarterly.csv",
}

OUTPUT_PATH = "../data/processed/master_dataset.csv"

df_master = build_master_panel(FILES, OUTPUT_PATH)


 Loading datasets and standardizing keys...
    Loaded population: (882, 6)
    Loaded immigration: (930, 13)
    Loaded interprov: (930, 8)
    Loaded cpi: (1890, 19)
    Loaded interest_rate: (704, 5)
    Loaded housing: (819, 10)

 Merging...
   + Joined immigration. New shape: (882, 15)
   + Joined interprov. New shape: (882, 19)
   + Joined cpi. New shape: (882, 34)
   + Joined interest_rate. New shape: (882, 35)
   + Joined housing. New shape: (882, 41)

 MASTER DATASET CREATED SUCCESSFULLY
 • Saved to: ../data/processed/master_dataset.csv
 • Gold Standard Saved to: ../data/processed/master_dataset.csv
 • Final Dimensions: (882, 41)

Top 5 Rows (Ontario):
    reference_period province  population quarter  year quarter_date  \
567          Q1 2010  Ontario    13059426      Q1  2010   2010-03-31   
568          Q2 2010  Ontario    13088924      Q2  2010   2010-06-30   
569          Q3 2010  Ontario    13136481      Q3  2010   2010-09-30   
570          Q4 2010  Ontario    13189987 

## Overview of the merged data

In [94]:
df_final = pd.read_csv("../data/processed/master_dataset.csv")

In [95]:
df_final

,reference_period,province,population,quarter,year,quarter_date,reference_period_immigration,immigrants,net_emigration,emigrants,...,Energy,Goods,Services,target_rate,total_housing_starts,single_detached_starts,multiples_starts,semi_detached_starts,row_housing_starts,apartment_starts
0,Q1 2010,Alberta,3702290,Q1,2010,2010-03-31,Q1 2010,6765.0,1313.0,1450.0,...,144.766667,109.133333,136.500000,0.250000,5651.0,3976.0,1675.0,749.0,436.0,490.0
1,Q2 2010,Alberta,3714999,Q2,2010,2010-06-30,Q2 2010,9732.0,1082.0,1731.0,...,138.933333,108.533333,137.466667,0.334615,8238.0,5580.0,2658.0,743.0,824.0,1091.0
2,Q3 2010,Alberta,3732104,Q3,2010,2010-09-30,Q3 2010,9485.0,825.0,2169.0,...,146.166667,108.566667,138.033333,0.765152,7437.0,4639.0,2798.0,683.0,565.0,1550.0
3,Q4 2010,Alberta,3747235,Q4,2010,2010-12-31,Q4 2010,6675.0,1167.0,1530.0,...,141.966667,108.566667,138.100000,1.000000,5762.0,3656.0,2106.0,562.0,771.0,773.0
4,Q1 2011,Alberta,3754422,Q1,2011,2011-03-31,Q1 2011,5917.0,1284.0,1617.0,...,155.433333,110.500000,138.500000,1.000000,4211.0,2462.0,1749.0,466.0,485.0,798.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877,Q3 2024,Yukon,47595,Q3,2024,2024-09-30,Q3 2024,276.0,-5.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
878,Q4 2024,Yukon,47939,Q4,2024,2024-12-31,Q4 2024,130.0,2.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
879,Q1 2025,Yukon,48176,Q1,2025,2025-03-31,Q1 2025,242.0,7.0,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
880,Q2 2025,Yukon,48089,Q2,2025,2025-06-30,Q2 2025,178.0,-1.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
# 1. Basic Shape and Columns
print(f"Dataset Shape: {df_final.shape}")
print("-" * 30)
print("Columns found:")
for col in df_final.columns:
    print(f" - {col}")

Dataset Shape: (882, 41)
------------------------------
Columns found:
 - reference_period
 - province
 - population
 - quarter
 - year
 - quarter_date
 - reference_period_immigration
 - immigrants
 - net_emigration
 - emigrants
 - returning_emigrants
 - net_temp_emigration
 - net_npr
 - npr_inflows
 - npr_outflows
 - reference_period_interprov
 - in_migrants
 - out_migrants
 - net_interprovincial_migration
 - All-items
 - Food
 - Shelter
 - Household operations, furnishings and equipment
 - Clothing and footwear
 - Transportation
 - Gasoline
 - Health and personal care
 - Recreation, education and reading
 - Alcoholic beverages, tobacco products and recreational cannabis
 - All-items excluding food and energy
 - All-items excluding energy
 - Energy
 - Goods
 - Services
 - target_rate
 - total_housing_starts
 - single_detached_starts
 - multiples_starts
 - semi_detached_starts
 - row_housing_starts
 - apartment_starts


### Missing Values by Column

In [97]:
missing_counts = df_final.isnull().sum()

# Filter to only show columns that actually have missing data
missing_data = missing_counts[missing_counts > 0]

if missing_data.empty:
    print("Great! No missing values found in any column.")
else:
    print("Columns with missing data:\n")
    print(missing_data.sort_values(ascending=False))
    
    # Optional: Print percentage
    print("\nPercentage Missing:")
    print((missing_data / len(df_final) * 100).round(2).astype(str) + "%")

Columns with missing data:

npr_inflows                                                        690
npr_outflows                                                       690
net_temp_emigration                                                570
Clothing and footwear                                              189
Services                                                           189
Recreation, education and reading                                  189
Alcoholic beverages, tobacco products and recreational cannabis    189
All-items excluding food and energy                                189
All-items excluding energy                                         189
Energy                                                             189
Goods                                                              189
total_housing_starts                                               189
target_rate                                                        189
Gasoline                                         

 ### Missing Values by Province

In [98]:
# Filter for rows where Shelter is missing and check the unique provinces
missing_provinces = df_final[df_final['Shelter'].isna()]['province'].unique()

print(f"Provinces with missing CPI data: {missing_provinces}")

Provinces with missing CPI data: ['Northwest Territories 5' 'Nunavut 5' 'Yukon']


In [99]:
# We count how many NaN values exist for each province across all columns
province_missing = df_final.groupby("province").apply(lambda x: x.isnull().sum().sum())

print("Total missing values per Province:")
print(province_missing[province_missing > 0].sort_values(ascending=False))

# Quick visual check: how many rows per province have *at least one* missing value?
rows_with_issues = df_final[df_final.isnull().any(axis=1)]["province"].value_counts()

if not rows_with_issues.empty:
    print("\nNumber of rows with incomplete data (by Province):")
    print(rows_with_issues)
else:
    print("\nNo incomplete rows found per province.")

Total missing values per Province:
province
Northwest Territories 5      2205
Nunavut 5                    2205
Yukon                        1527
Alberta                       141
British Columbia              141
Canada                        141
Manitoba                      141
New Brunswick                 141
Newfoundland and Labrador     141
Nova Scotia                   141
Ontario                       141
Prince Edward Island          141
Quebec                        141
Saskatchewan                  141
dtype: int64

Number of rows with incomplete data (by Province):
province
Alberta                      63
British Columbia             63
Canada                       63
Manitoba                     63
New Brunswick                63
Newfoundland and Labrador    63
Northwest Territories 5      63
Nova Scotia                  63
Nunavut 5                    63
Ontario                      63
Prince Edward Island         63
Quebec                       63
Saskatchewan          

In [100]:
print("\nMISSINGNESS BY PROVINCE (%)")
missing_by_province = (
    df_final.groupby("province")
         .apply(lambda df: df.isna().mean() * 100)
)
display(missing_by_province.round(2))


MISSINGNESS BY PROVINCE (%)


,reference_period,province,population,quarter,year,quarter_date,reference_period_immigration,immigrants,net_emigration,emigrants,...,Energy,Goods,Services,target_rate,total_housing_starts,single_detached_starts,multiples_starts,semi_detached_starts,row_housing_starts,apartment_starts
province,,,,,,,,,,,,,,,,,,,,,
Alberta,0.0,0.0,0.0,0.0,0.0,0.0,1.59,1.59,1.59,1.59,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
British Columbia,0.0,0.0,0.0,0.0,0.0,0.0,1.59,1.59,1.59,1.59,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Canada,0.0,0.0,0.0,0.0,0.0,0.0,1.59,1.59,1.59,1.59,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Manitoba,0.0,0.0,0.0,0.0,0.0,0.0,1.59,1.59,1.59,1.59,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
New Brunswick,0.0,0.0,0.0,0.0,0.0,0.0,1.59,1.59,1.59,1.59,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Newfoundland and Labrador,0.0,0.0,0.0,0.0,0.0,0.0,1.59,1.59,1.59,1.59,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Northwest Territories 5,0.0,0.0,0.0,0.0,0.0,0.0,100.00,100.00,100.00,100.00,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0
Nova Scotia,0.0,0.0,0.0,0.0,0.0,0.0,1.59,1.59,1.59,1.59,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Nunavut 5,0.0,0.0,0.0,0.0,0.0,0.0,100.00,100.00,100.00,100.00,...,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.0


## Data Cleaning


In [101]:
# 1. Remove Territories
# These regions often lack economic data (CPI/Housing).
territories = [
    "Northwest Territories 5", 
    "Nunavut 5", 
    "Yukon"
]
panel = df_final[~df_final["province"].isin(territories)].copy()

print(f"Territories dropped. Remaining provinces: {panel['province'].nunique()}")

# 2. Drop "Broken" Columns (>90% Missing)
# These are the columns where the  data is too sparse.
cols_to_drop = [
    "npr_inflows", 
    "npr_outflows", 
    "net_temp_emigration"
]

# Using errors='ignore' so it doesn't crash if you run this cell twice
panel = panel.drop(columns=cols_to_drop, errors='ignore')

print(f"Dropped {len(cols_to_drop)} sparse/broken columns.")

Territories dropped. Remaining provinces: 11
Dropped 3 sparse/broken columns.


In [102]:
# 1. Drop rows missing Immigration Data
# (We expect this to be small, usually ~1-2%)
imm_cols = [
    "immigrants", "net_emigration", "emigrants", 
    "returning_emigrants", "net_npr"
]

initial_rows = len(panel)
panel = panel.dropna(subset=imm_cols)
print(f"Dropped {initial_rows - len(panel)} rows missing immigration data.")

# 2. Drop rows missing CPI Data
# (CPI is our Target Variable, so we can't have NaNs here)
cpi_cols = [
    "All-items", "Food", "Shelter", "Energy", 
    "Gasoline", "Services", "Goods"
]

# We filter by the main categories to be safe
panel = panel.dropna(subset=cpi_cols)

print(f"Final Row Count: {len(panel)}")

Dropped 11 rows missing immigration data.
Final Row Count: 682


In [103]:
# Sort for presentation
panel = panel.sort_values(["province", "quarter_date"]).reset_index(drop=True)

# 1. Check Overall Missingness (Should be 0.00% mostly)
print("--- Missing Values (%) ---")
missing_check = (panel.isna().mean() * 100).round(2)
# Only show columns that still have issues
print(missing_check[missing_check > 0]) 

if missing_check.sum() == 0:
    print("Perfect! No missing values remaining.")

# 2. Check Dimensions
print(f"\nFinal Shape: {panel.shape}")

--- Missing Values (%) ---
Series([], dtype: float64)
Perfect! No missing values remaining.

Final Shape: (682, 38)


In [104]:
output_path = "../data/processed/master_dataset_clean.csv"

panel.to_csv(output_path, index=False)
print(f"Saved clean dataset to: {output_path}")

Saved clean dataset to: ../data/processed/master_dataset_clean.csv
